In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [5]:
# Load your trained model
model_path = r'C:\Users\Suyash Tambe\Desktop\Deepfakevideo\71%cnn_deepfake_best.keras'  
model = load_model(model_path)

In [6]:
# Define the preprocessing function based on your model requirements
def preprocess_frame(frame, target_size=(199, 199)):
    # Resize the frame to the target input size (e.g., 199x199)
    frame_resized = cv2.resize(frame, target_size)
    
    # Convert from BGR  to RGB if your model expects RGB
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    
    # Normalize the pixel values 
    frame_normalized = frame_rgb / 255.0
    
    return frame_normalized

# Function to detect deepfake frames and their timestamps
def detect_deepfake_frames(frame_predictions, fps=30):
    deepfake_frames = []
    
    # Go through the frame-level predictions
    for i, pred in enumerate(frame_predictions):
        if pred > 0.5:  # Assuming threshold of 0.5 for detecting fake
            timestamp = i / fps  # Calculate timestamp based on frame rate (fps)
            deepfake_frames.append((i, timestamp))  # Record frame index and timestamp
            
    return deepfake_frames

# Inference function to predict on a single video and detect frames
def predict_deepfake_on_video(video_path, model, sequence_length=100, fps=30):
    # Open the video using OpenCV
    video_capture = cv2.VideoCapture(video_path)
    frame_predictions = []
    
    # Initialize a list to hold the sequence of frames
    frame_sequence = []

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Preprocess the frame as required by your model
        preprocessed_frame = preprocess_frame(frame)
        frame_sequence.append(preprocessed_frame)
        
        # Check if we have enough frames to form a sequence 
        if len(frame_sequence) == sequence_length:
            # Convert frame sequence to a numpy array and add batch dimension
            frame_sequence_array = np.expand_dims(np.array(frame_sequence), axis=0)  # Shape: (1, 100, 199, 199, 3)
            
            # Predict on the sequence of frames
            preds = model.predict(frame_sequence_array)[0]  # Get predictions for all 100 frames
            frame_predictions.extend(preds)
            
            # Clear the frame sequence to start accumulating the next batch
            frame_sequence = []
    
    # Release video capture
    video_capture.release()
    
    # Get the frames where deepfake is detected
    deepfake_frames = detect_deepfake_frames(frame_predictions, fps=fps)
    
    if deepfake_frames:
        print(f"Deepfake detected at the following frames and timestamps:")
        for frame, timestamp in deepfake_frames:
            print(f"Frame: {frame}, Timestamp: {timestamp:.2f} seconds")
    else:
        print("No deepfake detected in this video.")
        
    return deepfake_frames


video_path = r'C:\Users\Suyash Tambe\Desktop\Deepfakevideo\train_samples\FAKE\bbhpvrmbse.mp4'  
fps = 30  # Set your video's frames per second

# Make predictions and detect deepfake frames
detected_frames = predict_deepfake_on_video(video_path, model, sequence_length=100, fps=fps)


1/1 ━━━━━━━━━━━━━━━━━━━━ 225s 225s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Deepfake detected at the following frames and timestamps:
Frame: 0, Timestamp: 0.00 seconds
Frame: 1, Timestamp: 0.03 seconds
Frame: 2, Timestamp: 0.07 seconds
